Устанавливаем требуемые пакеты

In [22]:
!pip install spacy-transformers
!python -m spacy download ru_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-09 10:15:29.265610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 43.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


Импортируем библиотеки

In [3]:
import pandas as pd
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.load("ru_core_news_sm")

Монтируем Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Задания/AdditionalAssignment'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Задания/AdditionalAssignment


Загружаем датасет

In [5]:
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding="utf8", sep='\t')

Подготавливаем данные

In [6]:
df = df.sample(frac=0.5)
df = df[df['sentiment'] != 'neautral']
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]
train_data = data[:15000]
valid_data = data[15000:]

Функция для формирования списка документов

In [7]:
def make_docs(data):
    docs = []
    a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

Создаем список документов для обучения

In [8]:
num_texts = 100

train_docs = make_docs(train_data[:num_texts])
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/100 [00:00<?, ?it/s]


Инициализируем конфигурацию

In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy init fill-config base_config.cfg config.cfg

2023-06-09 09:54:01.224153: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Пробуем обучить модель

In [20]:
!python -m spacy train config.cfg --gpu-id 0 --output ./output

2023-06-09 09:54:18.372770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-09 09:54:26,412] [INFO] Set up nlp object from config
[2023-06-09 09:54:26,429] [INFO] Pipeline: ['transformer', 'textcat_multilabel']
[2023-06-09 09:54:26,432] [INFO] Created vocabulary
[2023-06-09 09:54:26,434] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a 

Функция для тестирования модели

In [ ]:
import spacy

nlp = spacy.load("output/model-last")
text = ""
print("type : ‘quit’ to exit")
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")